In [2]:
import requests
import json
import time

myaddress = 'TW7eKyRw8ow98fpFibLe84qKwSbrvuD1KB' #no txn related to this address in mainnet so far
address_example = 'TUD4YXYdj2t1gP5th3A7t97mx1AUmrrQRt' #example address in tron offcial doc, can be used to retrieve sample txn from mainnet
address_DEX = 'TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX' #a D-exchange address, actively sending/recieving USDT on mainnet 
trc20_USDT = 'TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t'

In [3]:
def get_latest_block():
    #return the latest block number and ts on mainnet 
    url = "https://api.trongrid.io/v1/blocks/latest/events"
    response = requests.request("GET", url)

    block_number = json.loads(response.text)['data'][0]['block_number']
    block_ts = json.loads(response.text)['data'][0]['block_timestamp']
    
    return block_number, block_ts

get_latest_block()

(28180718, 1614933141000)

# 1.根据钱包地址

#### 获取账户详情

In [4]:
url_get_account = 'https://api.trongrid.io/v1/accounts/{address}'.format(address = address_example)
response = requests.request("GET", url_get_account)
print(json.loads(response.text)['data'][0].keys())

dict_keys(['owner_permission', 'account_resource', 'active_permission', 'address', 'create_time', 'latest_consume_time', 'allowance', 'latest_opration_time', 'free_asset_net_usageV2', 'is_witness', 'free_net_usage', 'assetV2', 'balance', 'trc20', 'latest_consume_free_time', 'account_name', 'latest_withdraw_time', 'witness_permission'])


#### 获取账户的历史交易

In [5]:
url_get_account_txn = 'https://api.trongrid.io/v1/accounts/{address}/transactions'.format(address = address_DEX)
response = requests.request("GET", url_get_account_txn)
print(json.loads(response.text)['data'][0])

{'ret': [{'contractRet': 'SUCCESS', 'fee': 2096500}], 'signature': ['731a5af1963dc8b9d905d21b2669fa75e345ea64413e93493d86439e578aa7f107c97872ff94e4e12a9ff1027f11cf3883b8f085ab380755c1706dc3b914da4f01'], 'txID': 'de07e4534bc446afafae6e695023e3efd49ef04cc77aa147ba4d3ffc9133c09d', 'net_usage': 0, 'raw_data_hex': '0a0200e722083481f3004e23a69040e0fc928c802f5aae01081f12a9010a31747970652e676f6f676c65617069732e636f6d2f70726f746f636f6c2e54726967676572536d617274436f6e747261637412740a15410583a68a3bcd86c25ab1bee482bac04a216b0261121541a614f803b6fd780986a42c78ec9c7f77e6ded13c2244a9059cbb00000000000000000000004124168572a717bb014fd979f583d50d2486586e4d0000000000000000000000000000000000000000000000000000000004c4b40070a2b98f8c802f900180ade204', 'net_fee': 48300, 'energy_usage': 1, 'blockNumber': 28180713, 'block_timestamp': 1614933126000, 'energy_fee': 2048200, 'energy_usage_total': 14631, 'raw_data': {'contract': [{'parameter': {'value': {'data': 'a9059cbb00000000000000000000004124168572a717bb014fd979f

#### 获取账户在某一合约下的历史交易

In [6]:
# 增加以下过滤条件，只获取trc20-USDT相关事件
url_get_account_txn_trc20 = 'https://api.trongrid.io/v1/accounts/{address}/transactions/trc20'.format(address = address_DEX)
querystring = {"contract_address":"TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t"}
response = requests.request("GET", url_get_account_txn_trc20, params=querystring)
print(json.loads(response.text)['data'][0])

{'transaction_id': 'de07e4534bc446afafae6e695023e3efd49ef04cc77aa147ba4d3ffc9133c09d', 'token_info': {'symbol': 'USDT', 'address': 'TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t', 'decimals': 6, 'name': 'Tether USD'}, 'block_timestamp': 1614933126000, 'from': 'TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX', 'to': 'TDG2Jmfbh7b1oEmkik1gFGMBKQTnjxHXFY', 'type': 'Transfer', 'value': '80000000'}


# 2. 根据合约地址

### 获取合约历史交易（暂无websocket订阅方法，只能GET）

In [7]:
url_get_contract_txn = "https://api.trongrid.io/v1/contracts/{c_address}/transactions".format(c_address = trc20_USDT)
#限制返回结果的最小区块时间戳，以及每页的交易属（默认为20，最大为200）
querystring = {"min_block_timestamp":"1614761300000", "limit":"200"} 
response = requests.request("GET", url_get_contract_txn, params = querystring)
print(json.loads(response.text)['data'][0])

{'ret': [{'contractRet': 'SUCCESS'}], 'signature': ['5059476943030d7836337357a03462fb4dbee4a58ff136987d8eed85eebf25be47ba177c3f9894ed6ddd26e2eb6e50a37079f56d7fb3807d532a1af5be8b7d7e01'], 'txID': '8797f872431dd1da484812b0b8413368216841425e318976d254ac334c4fca70', 'net_usage': 0, 'raw_data_hex': '0a0200e62208313be0ad1d7ccf854098e8f18d802f5aae01081f12a9010a31747970652e676f6f676c65617069732e636f6d2f70726f746f636f6c2e54726967676572536d617274436f6e747261637412740a154179309abcff2cf531070ca9222a1f72c4a5136874121541a614f803b6fd780986a42c78ec9c7f77e6ded13c2244a9059cbb000000000000000000000041010b9b1ae53560bb395c469b98f29a52c117644000000000000000000000000000000000000000000000000000000000048cb41070bdd1928c802f90018094ebdc03', 'net_fee': 48440, 'energy_usage': 1, 'block_timestamp': '1614933177000', 'blockNumber': '28180730', 'energy_fee': 2048200, 'energy_usage_total': 14631, 'raw_data': {'contract': [{'parameter': {'value': {'data': 'a9059cbb000000000000000000000041010b9b1ae53560bb395c469b98f29a52c

In [8]:
#估算:最新的trc20-USDT交易  大约分布在10个区块中（大约需要三秒钟产生一个新的区块）
set([i['block_timestamp'] for i in json.loads(response.text)['data']])

{'1614933147000',
 '1614933150000',
 '1614933153000',
 '1614933156000',
 '1614933159000',
 '1614933162000',
 '1614933165000',
 '1614933168000',
 '1614933171000',
 '1614933174000',
 '1614933177000'}

### 获取合约历史Transfer事件

In [9]:
url_get_contract_events = "https://api.trongrid.io/v1/contracts/{c_address}/events".format(c_address = trc20_USDT)
#限制返回结果的最小区块时间戳，以及每页的交易属（默认为20，最大为200）
querystring = {"event_name":"Transfer","block_number":get_latest_block()[0],"limit":"200"}
response = requests.request("GET", url_get_contract_txn, params = querystring)
print(querystring['block_number'])
print(json.loads(response.text)['data'][0])

28180753
{'ret': [{'contractRet': 'SUCCESS'}], 'signature': ['241ff9b08b228f5ad9b82942d334a6ecb60a4282956b69d8f68bcda45d9fa163d13b6e347c79ab9f830dad999bd3b1caf7bf929de0c32360bebeaeb8640ae6c001'], 'txID': '285d2d35e72125343e3e3cc27e992f786ef51afe6e2018fcaafc81c1f05bfe94', 'net_usage': 345, 'raw_data_hex': '0a0200fc2208fc2354ad57bd5f2040e88e9a8c802f5aae01081f12a9010a31747970652e676f6f676c65617069732e636f6d2f70726f746f636f6c2e54726967676572536d617274436f6e747261637412740a15419a51946b3b930fedcac65cc63f657508b178bbc6121541a614f803b6fd780986a42c78ec9c7f77e6ded13c2244a9059cbb000000000000000000000000eede73141a423de078a9e40c4d0531d1612e17b700000000000000000000000000000000000000000000000000000000082a744070dddc968c802f900180c2d72f', 'net_fee': 0, 'energy_usage': 0, 'block_timestamp': '1614933249000', 'blockNumber': '28180754', 'energy_fee': 2048340, 'energy_usage_total': 14631, 'raw_data': {'contract': [{'parameter': {'value': {'data': 'a9059cbb000000000000000000000000eede73141a423de078a9e40c4d05

In [37]:
len(json.loads(response.text)['data'])

200

In [11]:
#以mainnet区块blockNumber 28180730 为例，其中的USDT交易仅有23笔
len([i['blockNumber'] for i in json.loads(response.text)['data'] if i['blockNumber'] == str(querystring['block_number'])])

23

In [12]:
# 若声明每页的返回数为200且最新区块USDT交易不足200，则会返回之前区块的USDT交易信息
set([i['blockNumber'] for i in json.loads(response.text)['data']])

{'28180744',
 '28180745',
 '28180746',
 '28180747',
 '28180748',
 '28180749',
 '28180750',
 '28180751',
 '28180752',
 '28180753',
 '28180754'}

### 获取某一代币所有持有者的余额信息

In [21]:
url_get_tec20_balances = "https://api.trongrid.io/v1/contracts/{c_address}/tokens".format(c_address = trc20_USDT)
querystring = {"limit":"200"} 
response = requests.request("GET", url_get_tec20_balances, params = querystring)
print(json.loads(response.text)['data'][:10])

[{'TMuA6YqfCeX8EhbfYEg5y7S4DqzSJireY9': '2500000000000000'}, {'TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX': '1625246835257550'}, {'TNaRAoLUyYEV2uF7GUrzSjRQTU8v5ZJ5VR': '1114538033737597'}, {'TWd4WrZ9wn84f5x1hZhL4DHvk738ns5jwb': '700000000000000'}, {'TFsmkjujT9omG5TFrEXYbDocTdjeeHgcrQ': '600000007000124'}, {'TNMDwbHyAQo38tK4fCYe4ooSZXesbK8ExZ': '578000000000076'}, {'TM1zzNDZD2DPASbKcgdVoTYhfmYgtfwx9R': '440758301069313'}, {'TJyf3bMf5CXYe511EFPbN5dZ2W8edMnzsn': '420817281000042'}, {'TKHuVq1oKVruCGLvqVexFs6dawKv6fQgFs': '382181363328931'}, {'TXFBqBbqJommqZf7BV8NNYzePh97UmJodJ': '329928437497811'}]


CreateAccount
在链上创建账号. 一个已经激活的账号创建一个新账号需要花费 0.1 TRX 或等值 Bandwidth.
[https://cn.developers.tron.network/reference#%E5%9C%B0%E5%9D%80%E5%92%8C%E5%B8%90%E6%88%B7]